# Modelos de Cáculo

## CFG

In [ ]:
import nltk
from nltk import CFG

def probar_gramatica():
    # Definir la gramática libre de contexto
    gramatica = CFG.fromstring("""
    S -> A | B
    A -> "0" A "2" | "1" B "2" | ""
    B -> "1" B "2" | ""
    """)

    # Crear el parser para la gramática
    parser = nltk.ChartParser(gramatica)

    # Definir una oración de ejemplo
    oracion = "0122"
    palabras = oracion.split()  # Asegúrate de que las palabras estén separadas

    # Intentar analizar la oración
    print(f"Probando la oración: {oracion}")
    for arbol in parser.parse(palabras):
        print(arbol)



probar_gramatica()


In [ ]:
import nltk
from nltk import CFG
import itertools

def generar_cadenas(gramatica, simbolo_inicial='S', max_longitud=5):
    """
    Genera todas las strings posibles de la gramática hasta una longitud máxima.
    
    :param gramatica: La gramática en formato CFG.
    :param simbolo_inicial: El símbolo inicial (por defecto 'S').
    :param max_longitud: La longitud máxima de las strings generadas.
    :return: Una lista con todas las strings posibles.
    """
    producciones = gramatica.productions()
    print(producciones)
    def es_terminal(simbolo):
        """Verifica si un símbolo es terminal."""
        return simbolo in gramatica._lexical_index
    
    def generar_desde(simbolo, longitud_actual=0):
        """Genera recursivamente las cadenas a partir de un símbolo no terminal."""
        if es_terminal(simbolo):
            return [[simbolo]]  # Devuelve la cadena terminal como lista

        if longitud_actual >= max_longitud:
            return []  # No generar más si se ha alcanzado la longitud máxima

        cadenas = []
        for produccion in producciones:
            if produccion.lhs() == simbolo:
                partes = produccion.rhs()
                subcadenas = [generar_desde(s, longitud_actual + 1) for s in partes]
                for combinacion in itertools.product(*subcadenas):
                    cadenas.append([item for sublista in combinacion for item in sublista])
        return cadenas
    
    # Generar todas las cadenas desde el símbolo inicial
    todas_cadenas = generar_desde(gramatica.start())
    return [' '.join(cadena) for cadena in todas_cadenas]

# Definir la gramática libre de contexto
gramatica = CFG.fromstring("""
S -> A | B
A -> '0' A '2' | '1' B '2' | ""
B -> '1' B '2' | ""
""")

# Generar cadenas hasta longitud 6
cadenas_generadas = generar_cadenas(gramatica, max_longitud=39)

# Imprimir las cadenas generadas
for cadena in cadenas_generadas:
    ceros = cadena.count('0')
    unos = cadena.count('1')
    doses = cadena.count('2')
    print(f'{ceros} + {unos} =? {doses}', ceros + unos == doses)
    print(f"Cadena generada: '{cadena}'")


## Representación Gráfica palabras Generadas

In [ ]:
import nltk
from nltk import CFG
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import Counter

def generar_cadenas(gramatica, simbolo_inicial='S', max_longitud=5):
    """
    Genera todas las strings posibles de la gramática hasta una longitud máxima.
    
    :param gramatica: La gramática en formato CFG.
    :param simbolo_inicial: El símbolo inicial (por defecto 'S').
    :param max_longitud: La longitud máxima de las strings generadas.
    :return: Una lista con todas las strings posibles.
    """
    producciones = gramatica.productions()
    
    def es_terminal(simbolo):
        """Verifica si un símbolo es terminal."""
        return isinstance(simbolo, str) and simbolo in gramatica._lexical_index
    
    def generar_desde(simbolo, longitud_actual=0):
        """Genera recursivamente las cadenas a partir de un símbolo no terminal."""
        if es_terminal(simbolo):
            return [[simbolo]]  # Devuelve la cadena terminal como lista

        if longitud_actual >= max_longitud:
            return []  # No generar más si se ha alcanzado la longitud máxima

        cadenas = []
        for produccion in producciones:
            if produccion.lhs() == simbolo:
                partes = produccion.rhs()
                longitud_nueva = longitud_actual + sum(1 for s in partes if es_terminal(s))
                if longitud_nueva > max_longitud:
                    continue  # No continuar si la longitud de la cadena excede el límite
                subcadenas = [generar_desde(s, longitud_actual + 1) for s in partes]
                for combinacion in itertools.product(*subcadenas):
                    cadena_generada = [item for sublista in combinacion for item in sublista]
                    if len(cadena_generada) <= max_longitud:
                        cadenas.append(cadena_generada)
        return cadenas
    
    # Generar todas las cadenas desde el símbolo inicial
    todas_cadenas = generar_desde(gramatica.start())
    cadenas_unicas = set([' '.join(cadena) for cadena in todas_cadenas])
    return list(cadenas_unicas)

def graficar_distribucion_palabras_3d(cadenas_generadas):
    """
    Grafica una dispersión de las palabras generadas en el espacio tridimensional (ceros, unos, doses).
    
    :param cadenas_generadas: Lista de cadenas generadas.
    """
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(111, projection='3d')

    ceros = []
    unos = []
    doses = []

    for cadena in cadenas_generadas:
        ceros.append(cadena.count('0'))
        unos.append(cadena.count('1'))
        doses.append(cadena.count('2'))
    
    scatter = ax.scatter(ceros, unos, doses, c=doses, cmap='viridis')
    ax.set_xlabel('Cantidad de 0s')
    ax.set_ylabel('Cantidad de 1s')
    ax.set_zlabel('Cantidad de 2s')
    ax.set_title('Dispersión de la Distribución de las Palabras Generadas')
    plt.tight_layout()
    plt.show()

# Definir la gramática libre de contexto
gramatica = CFG.fromstring("""
S -> A | B
A -> '0' A '2' | '1' B '2' | ""
B -> '1' B '2' | ""
""")

# Generar cadenas hasta longitud 6
cadenas_generadas = generar_cadenas(gramatica, max_longitud=100)

# Imprimir las cadenas generadas
# for cadena in cadenas_generadas:
#     ceros = cadena.count('0')
#     unos = cadena.count('1')
#     doses = cadena.count('2')
#     print(f'{ceros} + {unos} =? {doses}', ceros + unos == doses)
#     print(f"Cadena generada: '{cadena}'")

# Graficar la distribución de las palabras generadas
graficar_distribucion_palabras(cadenas_generadas)


In [ ]:
import nltk
from nltk import CFG
import itertools
import plotly.graph_objects as go
from collections import Counter
import numpy as np

def generar_cadenas(gramatica, simbolo_inicial='S', max_longitud=5):
    """
    Genera todas las strings posibles de la gramática hasta una longitud máxima.
    
    :param gramatica: La gramática en formato CFG.
    :param simbolo_inicial: El símbolo inicial (por defecto 'S').
    :param max_longitud: La longitud máxima de las strings generadas.
    :return: Una lista con todas las strings posibles.
    """
    producciones = gramatica.productions()
    
    def es_terminal(simbolo):
        """Verifica si un símbolo es terminal."""
        return isinstance(simbolo, str) and simbolo in gramatica._lexical_index
    
    def generar_desde(simbolo, longitud_actual=0):
        """Genera recursivamente las cadenas a partir de un símbolo no terminal."""
        if es_terminal(simbolo):
            return [[simbolo]]  # Devuelve la cadena terminal como lista

        if longitud_actual >= max_longitud:
            return []  # No generar más si se ha alcanzado la longitud máxima

        cadenas = []
        for produccion in producciones:
            if produccion.lhs() == simbolo:
                partes = produccion.rhs()
                longitud_nueva = longitud_actual + sum(1 for s in partes if es_terminal(s))
                if longitud_nueva > max_longitud:
                    continue  # No continuar si la longitud de la cadena excede el límite
                subcadenas = [generar_desde(s, longitud_actual + 1) for s in partes]
                for combinacion in itertools.product(*subcadenas):
                    cadena_generada = [item for sublista in combinacion for item in sublista]
                    if len(cadena_generada) <= max_longitud:
                        cadenas.append(cadena_generada)
        return cadenas
    
    # Generar todas las cadenas desde el símbolo inicial
    todas_cadenas = generar_desde(gramatica.start())
    cadenas_unicas = set([' '.join(cadena) for cadena in todas_cadenas])
    return list(cadenas_unicas)

def graficar_distribucion_palabras_todas_iteraciones(cadenas_generadas_por_iteracion):
    """
    Grafica una dispersión de las palabras generadas en el espacio tridimensional (ceros, unos, doses) para todas las iteraciones usando Plotly.
    
    :param cadenas_generadas_por_iteracion: Lista de listas, donde cada sublista contiene las cadenas generadas en una iteración.
    """
    fig = go.Figure()

    colores = np.linspace(0, 1, len(cadenas_generadas_por_iteracion))
    
    # Invertir el orden de las iteraciones para plotear desde la última hasta la primera
    for idx in reversed(range(len(cadenas_generadas_por_iteracion))):
        cadenas_generadas = cadenas_generadas_por_iteracion[idx]
        ceros = []
        unos = []
        doses = []

        for cadena in cadenas_generadas:
            ceros.append(cadena.count('0'))
            unos.append(cadena.count('1'))
            doses.append(cadena.count('2'))
        
        fig.add_trace(go.Scatter3d(
            x=ceros,
            y=unos,
            z=doses,
            mode='markers',
            marker=dict(
                size=5,
                color=colores[idx],  # Color diferente para cada iteración
                colorscale='Viridis',
                opacity=0.8
            ),
            name=f'Iteración {idx + 1}'
        ))
    
    fig.update_layout(
        scene=dict(
            xaxis_title='Cantidad de 0s',
            yaxis_title='Cantidad de 1s',
            zaxis_title='Cantidad de 2s'
        ),
        title='Dispersión de la Distribución de las Palabras Generadas en Todas las Iteraciones'
    )
    fig.show(renderer='browser')  # Cambiar el renderer a 'browser' para evitar problemas de visualización en algunos entornos


# 0^a 1^b 2^(a+b)
l1 = """
S -> A | B
A -> '0' A '2' | '1' B '2' | ""
B -> '1' B '2' | ""
"""

# 0^a 1^a

l2 = """
S -> '0' S '1' | ""
"""

# jerarquia aritmética

l3 = """
S -> '0' | '1' | '2' | S '+' S | S '-' S | S '*' S | S '/' S | '('S')' | ""
"""
# Lista de gramáticas a utilizar
gramaticas = {
    'l1': """
    S -> A | B
    A -> '0' A '2' | '1' B '2' | ""
    B -> '1' B '2' | ""
    """,
    'l2': """
    S -> '0' S '1' | ""
    """,
    'l3': """
    S -> '0' | '1' | '2' | S '+' S | S '-' S | S '*' S | S '/' S | '(' S ')'
    """,
    # 1. Gramática para palíndromos sobre el alfabeto {0,1,2}
    'palindromo': """
    S -> '0' S '0' | '1' S '1' | '2' S '2' | '0' | '1' | '2' | ""
    """,
    # 2. Gramática para L = { 0^n 2 1^n | n ≥ 1 }
    'balanced': """
    S -> '0' S '1' | '0' '2' '1' | '0' '1'
    """,
    # 3. Gramática donde el número de '0's es el doble de '1's
    'doble_ceros': """
    S -> '0' '0' S '1' | '0' '0' '1'
    """,
    # 4. Gramática para cadenas donde el número de '2's es igual a la suma de '0's y '1's
    'suma_indices': """
    S -> A B
    A -> '0' A '2' | ""
    B -> '1' B '2' | ""
    """,
    # 5. Gramática para cadenas con número igual de '0's, '1's y '2's (aproximación)
    'igual_cantidad': """
    S -> '0' S '2' | A
    A -> '0' '1' '2' | '0' '1' '2' A
    """,
    # 6. Gramática para expresiones aritméticas con '0', '1', '2'
    'expresiones': """
    S -> S '+' M | S '-' M | M
    M -> M '*' T | M '/' T | T
    T -> '0' | '1' | '2' | '(' S ')'
    """,
    # 7. Gramática donde las '2's están enmarcadas por '0's y '1's alternados
    'alternado': """
    S -> '0' S '2' | '1' S '2' | '0' '2' | '1' '2'
    """,
    # 8. Gramática para números en base 3
    'base3': """
    S -> D S | D
    D -> '0' | '1' | '2'
    """,
    # 9. Gramática para cadenas donde la cantidad de '1's es igual a la suma de '0's y '2's
    'patron_especial': """
    S -> '0' S '1' | '2' S '1' | ""
    """
}

# Definir la gramática libre de contexto
gramatica = CFG.fromstring(l1)

# Generar y acumular las cadenas en cada iteración
generar_hasta = 5
cadenas_generadas_por_iteracion = []

for n in range(1, generar_hasta + 1):
    cadenas_generadas = generar_cadenas(gramatica, max_longitud=n)
    cadenas_generadas_por_iteracion.append(cadenas_generadas)
    
    # Imprimir las cadenas generadas
    # print(f"Iteración {n}", cadenas_generadas)
    # for cadena in cadenas_generadas:
    #     ceros = cadena.count('0')
    #     unos = cadena.count('1')
    #     doses = cadena.count('2')
    #     print(f'{ceros} + {unos} =? {doses}', ceros + unos == doses)
    #     print(f"Cadena generada: '{cadena}'")

# Graficar la distribución de las letras en las palabras generadas para todas las iteraciones
graficar_distribucion_palabras_todas_iteraciones(cadenas_generadas_por_iteracion)
